1. 데이터 불러오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/content/drive/MyDrive/TAVE프로젝트/data/최종_데이터셋7(월,일,색상,RGB).csv')
columns_needed_1=['꽃','월','일자','계절','꽃말','설명','이미지','색상','선택한 색상']
df = df[columns_needed_1]

palette = pd.read_csv('/content/drive/MyDrive/TAVE프로젝트/data/웹크롤링_색상팔레트.csv')
columns_needed_2 = ['c0','c1','c2','c3']
palette = palette[columns_needed_2]

In [4]:
df.head()

,꽃,월,일자,계절,꽃말,설명,이미지,색상,선택한 색상
0,각시붓꽃,3,NaN,봄,"부끄러움, 세련됨",각시라 하면 이제 막 시집 온 새색시를 연상케 한다. 그래서인지 여러가지 붓꽃들 중...,https://www.nihhs.go.kr/user/AttachFiles/FLOWF...,보라,"[125, 79, 163]"
1,감국,10,NaN,가을,그윽한 향기,가을 산야는 국화과 식물들 차지다. 특히 노란 꽃으로 향기까지 일품인 감국은 보는 ...,https://www.nihhs.go.kr/user/AttachFiles/FLOWF...,노랑,"[165, 164, 23]"
2,개나리,3,NaN,봄,"희망,깊은 정, 달성","우리나라 전역에서 봄 소식을 가장 먼저 알려주는 대표적인 꽃, 개나리는 어렵지 않게...",https://file.honestflower.kr/media/images/ingr...,노랑,"[213, 182, 54]"
3,개나리,4,1.0,봄,희망,"개나리, 봄을 알리는 전령사 하면 가장 먼저 떠오르는 꽃이다. ‘나리나리 개나리 잎...",https://file.honestflower.kr/media/images/ingr...,노랑,"[214, 182, 54]"
4,갯개미취,9,9.0,가을,추억,옛날 일 따위는 깨끗이 잊는 사람들이 많은 가운데서도 당신은 옛 일을 어제 일처럼 ...,https://lh6.googleusercontent.com/proxy/yRZlN4...,보라,"[209, 174, 248]"


In [5]:
palette.head()

,c0,c1,c2,c3
0,"[34, 40, 49]","[57, 62, 70]","[0, 173, 181]","[238, 238, 238]"
1,"[227, 253, 253]","[203, 241, 245]","[166, 227, 233]","[113, 201, 206]"
2,"[249, 247, 247]","[219, 226, 239]","[63, 114, 175]","[17, 45, 78]"
3,"[255, 199, 199]","[255, 226, 226]","[246, 246, 246]","[135, 133, 162]"
4,"[255, 245, 228]","[255, 227, 225]","[255, 209, 209]","[255, 148, 148]"


2. 가장 유사한 rgb 값으로 팔레트 찾기

In [6]:
import pandas as pd
import numpy as np
import re

# 유클리드 거리 계산 함수
def euclidean_distance(color1, color2):
    return np.sqrt(np.sum((color1 - color2) ** 2))

# '선택한 색상'과 가장 유사한 색상을 가진 팔레트 행을 찾는 함수
def find_closest_palette_row(selected_colors, palette, df_index):
    closest_rows = []

    for selected_color in selected_colors:
        selected_color = np.array(extract_numbers(selected_color)).astype(float)  # 색상 값을 배열로 변환하고 float 타입으로 변환
        min_distance = float('inf')
        closest_row = None
        closest_col = None

        for idx, row in palette.iterrows():
            for col in ['c0', 'c1', 'c2', 'c3']:
                palette_color = np.array(extract_numbers(row[col])).astype(float)
                distance = euclidean_distance(selected_color, palette_color)

                if distance < min_distance:
                    min_distance = distance
                    closest_row = row.copy()  # 행의 복사본을 생성하여 closest_row에 할당
                    closest_col = col  # 가장 가까운 팔레트 열 업데이트

        closest_row['유사 색상'] = closest_col  # '유사 색상' 열 추가
        closest_rows.append(closest_row)

    return pd.DataFrame(closest_rows, index=df_index)  # df의 인덱스 사용


# 문자열에서 숫자만 추출하는 함수
def extract_numbers(s):
    return list(map(int, re.findall(r'\d+', s)))

# '선택한 색상'에 대해 가장 유사한 팔레트 행을 찾기
selected_colors = df['선택한 색상']
closest_palette_rows = find_closest_palette_row(selected_colors, palette, df.index)

# df와 closest_palette_rows를 옆으로 합치기
merged_df = pd.concat([df, closest_palette_rows], axis=1)

print(merged_df)


        꽃   월    일자  계절           꽃말  \
0    각시붓꽃   3   NaN   봄    부끄러움, 세련됨   
1      감국  10   NaN  가을       그윽한 향기   
2     개나리   3   NaN   봄  희망,깊은 정, 달성   
3     개나리   4   1.0   봄           희망   
4    갯개미취   9   9.0  가을           추억   
..    ...  ..   ...  ..          ...   
827   흰앵초   3  26.0   봄          첫사랑   
828  흰제비꽃   1   6.0  겨울     순진무구한 사랑   
829  히아신스   3   NaN   봄   마음의 기쁨, 승리   
830  히야신스   1   4.0  겨울       차분한 사랑   
831   히어리   3   9.0   봄        봄의 노래   

                                                    설명  \
0    각시라 하면 이제 막 시집 온 새색시를 연상케 한다. 그래서인지 여러가지 붓꽃들 중...   
1    가을 산야는 국화과 식물들 차지다. 특히 노란 꽃으로 향기까지 일품인 감국은 보는 ...   
2    우리나라 전역에서 봄 소식을 가장 먼저 알려주는 대표적인 꽃, 개나리는 어렵지 않게...   
3    개나리, 봄을 알리는 전령사 하면 가장 먼저 떠오르는 꽃이다. ‘나리나리 개나리 잎...   
4    옛날 일 따위는 깨끗이 잊는 사람들이 많은 가운데서도 당신은 옛 일을 어제 일처럼 ...   
..                                                 ...   
827  첫사랑의 추억때문에 현실의 사랑을 바라보지 못하고 있지는 않습니까? 아련한 추억은 ...   
828        당신은 내성적이고 고상한 매력이 있지만 은밀한 사랑에 쉽게 빠지는 사람입니다. 

3. 해당 팔레트 및 유사 색상 추출 데이터와 기존 데이터 병합

In [7]:
merged_df

,꽃,월,일자,계절,꽃말,설명,이미지,색상,선택한 색상,c0,c1,c2,c3,유사 색상
0,각시붓꽃,3,NaN,봄,"부끄러움, 세련됨",각시라 하면 이제 막 시집 온 새색시를 연상케 한다. 그래서인지 여러가지 붓꽃들 중...,https://www.nihhs.go.kr/user/AttachFiles/FLOWF...,보라,"[125, 79, 163]","[251, 250, 205]","[222, 186, 206]","[186, 148, 209]","[127, 102, 157]",c3
1,감국,10,NaN,가을,그윽한 향기,가을 산야는 국화과 식물들 차지다. 특히 노란 꽃으로 향기까지 일품인 감국은 보는 ...,https://www.nihhs.go.kr/user/AttachFiles/FLOWF...,노랑,"[165, 164, 23]","[250, 248, 241]","[250, 234, 177]","[229, 186, 115]","[197, 137, 64]",c3
2,개나리,3,NaN,봄,"희망,깊은 정, 달성","우리나라 전역에서 봄 소식을 가장 먼저 알려주는 대표적인 꽃, 개나리는 어렵지 않게...",https://file.honestflower.kr/media/images/ingr...,노랑,"[213, 182, 54]","[28, 10, 0]","[54, 21, 0]","[96, 54, 1]","[204, 149, 68]",c3
3,개나리,4,1.0,봄,희망,"개나리, 봄을 알리는 전령사 하면 가장 먼저 떠오르는 꽃이다. ‘나리나리 개나리 잎...",https://file.honestflower.kr/media/images/ingr...,노랑,"[214, 182, 54]","[28, 10, 0]","[54, 21, 0]","[96, 54, 1]","[204, 149, 68]",c3
4,갯개미취,9,9.0,가을,추억,옛날 일 따위는 깨끗이 잊는 사람들이 많은 가운데서도 당신은 옛 일을 어제 일처럼 ...,https://lh6.googleusercontent.com/proxy/yRZlN4...,보라,"[209, 174, 248]","[12, 19, 79]","[29, 38, 125]","[92, 70, 156]","[212, 173, 252]",c3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
827,흰앵초,3,26.0,봄,첫사랑,첫사랑의 추억때문에 현실의 사랑을 바라보지 못하고 있지는 않습니까? 아련한 추억은 ...,https://digitalchosun.dizzo.com/site/data/img_...,초록,"[208, 224, 221]","[252, 209, 209]","[236, 226, 225]","[211, 224, 220]","[174, 225, 225]",c2
828,흰제비꽃,1,6.0,겨울,순진무구한 사랑,당신은 내성적이고 고상한 매력이 있지만 은밀한 사랑에 쉽게 빠지는 사람입니다.,https://img1.daumcdn.net/thumb/R1280x0.fjpg/?f...,흰색,"[236, 238, 230]","[28, 103, 88]","[61, 131, 97]","[214, 205, 164]","[238, 242, 230]",c3
829,히아신스,3,NaN,봄,"마음의 기쁨, 승리",꽃에는 늘 전설이 있게 마련이다. 히야신스는 그리이스의 아름다운 청년 Hyacint...,https://upload.wikimedia.org/wikipedia/commons...,보라,"[121, 31, 240]","[0, 0, 0]","[82, 5, 123]","[137, 44, 220]","[188, 111, 241]",c2
830,히야신스,1,4.0,겨울,차분한 사랑,봄이 찾아올 때마다 포근한 향기와 함께 피는 히야신스를 탄생화로 갖는 당신은 상냥하...,https://encrypted-tbn1.gstatic.com/images?q=tb...,보라,"[43, 22, 122]","[12, 19, 79]","[29, 38, 125]","[92, 70, 156]","[212, 173, 252]",c1


4. 유사 색상을 제외한 3가지 색상 중, 2개를 선택해서, 해당 rgb와 가장 유사한 색을 가진 꽃1, 꽃2 열 생성

In [8]:
import pandas as pd
import numpy as np
from scipy.spatial import distance
import re  # 정규 표현식 모듈 추가
import random

In [9]:
import re
import numpy as np
from scipy.spatial import distance
import pandas as pd

# 유사 색상을 제외한 3가지 색상 중, 2개를 선택하는 함수
def select_two_colors(row):
    colors = [row[col] for col in ['c0', 'c1', 'c2', 'c3'] if col != row['유사 색상']]
    return colors[:2]

# RGB 문자열에서 숫자만 추출하는 함수
def extract_numbers(color_str):
    return list(map(int, re.findall(r'\d+', color_str)))

# RGB 색상 간의 유클리드 거리 계산 함수
def euclidean_distance(color1, color2):
    return distance.euclidean(color1, color2)

# 가장 유사한 꽃을 찾는 함수
def find_similar_flowers(merged_df):
    results = []

    for index, row in merged_df.iterrows():
        selected_colors = select_two_colors(row)
        color1 = np.array(extract_numbers(selected_colors[0])).astype(float)
        color2 = np.array(extract_numbers(selected_colors[1])).astype(float)

        min_distance1 = float('inf')
        min_distance2 = float('inf')
        best_match1 = None
        best_match2 = None

        for idx, comp_row in merged_df.iterrows():
            if comp_row['꽃'] == row['꽃']:
                continue

            comp_color = np.array(extract_numbers(comp_row['선택한 색상'])).astype(float)

            distance1 = euclidean_distance(color1, comp_color)
            distance2 = euclidean_distance(color2, comp_color)

            if distance1 < min_distance1:
                min_distance1 = distance1
                best_match1 = comp_row

            if distance2 < min_distance2:
                min_distance2 = distance2
                best_match2 = comp_row

        if best_match1 is not None and best_match2 is not None:
            if best_match1['꽃'] == best_match2['꽃']:
                replace_flower = merged_df[(merged_df['선택한 색상'] == '[255, 255, 255]') &
                                           (merged_df['꽃'] != row['꽃']) &
                                           (merged_df['꽃'] != best_match1['꽃'])]
                if not replace_flower.empty:
                    replace_row = replace_flower.sample(n=1).iloc[0]
                    best_match2 = replace_row

            if best_match1['꽃'] != best_match2['꽃']:
                results.append((best_match1['꽃'], best_match1['이미지'], best_match2['꽃'], best_match2['이미지']))

    return results

# 결과에서 꽃 이미지를 찾는 함수
def find_flower_images(merged_df, results):
    flower1_images = []
    flower2_images = []

    for result in results:
        flower1_images.append(result[1])
        flower2_images.append(result[3])

    return flower1_images, flower2_images

# 데이터프레임의 컬럼 이름 확인
print(merged_df.columns)

# 새로운 데이터 프레임 생성
results = find_similar_flowers(merged_df)
flower1_images, flower2_images = find_flower_images(merged_df, results)

# 새로운 데이터 프레임을 만듭니다.
new_df = pd.DataFrame({
    '꽃1': [result[0] for result in results],
    '이미지1': flower1_images,
    '꽃2': [result[2] for result in results],
    '이미지2': flower2_images
})

# 새로운 데이터 프레임 출력
print(new_df)


Index(['꽃', '월', '일자', '계절', '꽃말', '설명', '이미지', '색상', '선택한 색상', 'c0', 'c1',
       'c2', 'c3', '유사 색상'],
      dtype='object')
               꽃1                                               이미지1  \
0          문스톤 장미  https://file.honestflower.kr/media/images/ingr...   
1             산자고  https://cdn.cctoday.co.kr/news/photo/201103/59...   
2         블랙뷰티 장미  https://file.honestflower.kr/media/images/ingr...   
3         블랙뷰티 장미  https://file.honestflower.kr/media/images/ingr...   
4            히야신스  https://encrypted-tbn1.gstatic.com/images?q=tb...   
..            ...                                                ...   
827  웨스트민스터 애비 장미  https://file.honestflower.kr/media/images/ingr...   
828            팬지  https://encrypted-tbn0.gstatic.com/images?q=tb...   
829       블랙뷰티 장미  https://file.honestflower.kr/media/images/ingr...   
830            팬지  https://encrypted-tbn0.gstatic.com/images?q=tb...   
831       피치팡팡 장미  https://file.honestflower.kr/media/images/ingr...   

        

In [10]:
new_df


,꽃1,이미지1,꽃2,이미지2
0,문스톤 장미,https://file.honestflower.kr/media/images/ingr...,스카비오사,https://file.honestflower.kr/media/images/ingr...
1,산자고,https://cdn.cctoday.co.kr/news/photo/201103/59...,로사넬라 장미,https://file.honestflower.kr/media/images/ingr...
2,블랙뷰티 장미,https://file.honestflower.kr/media/images/ingr...,흰색동백,https://cdn.011st.com/11dims/resize/600x600/qu...
3,블랙뷰티 장미,https://file.honestflower.kr/media/images/ingr...,옥잠화,https://yummygarden.co.kr/web/product/big/2019...
4,히야신스,https://encrypted-tbn1.gstatic.com/images?q=tb...,수영,https://www.cnews.or.kr/news/photo/202108/1121...
...,...,...,...,...
827,웨스트민스터 애비 장미,https://file.honestflower.kr/media/images/ingr...,설악초,https://www.nihhs.go.kr/user/AttachFiles/FLOWF...
828,팬지,https://encrypted-tbn0.gstatic.com/images?q=tb...,무스카리,https://file.honestflower.kr/media/images/ingr...
829,블랙뷰티 장미,https://file.honestflower.kr/media/images/ingr...,팬지,https://encrypted-tbn0.gstatic.com/images?q=tb...
830,팬지,https://encrypted-tbn0.gstatic.com/images?q=tb...,제비꽃,https://www.nihhs.go.kr/user/AttachFiles/FLOWF...


5. 꽃, 꽃1, 꽃2 가 서로 다른지 확인

In [11]:
# '꽃1' 또는 '꽃2' 컬럼에 결측치가 있는 행을 찾습니다.
missing_values = new_df[new_df['꽃1'].isna() | new_df['꽃2'].isna()]

# 결측치가 있는 행이 있는지 확인하고 출력합니다.
if not missing_values.empty:
    print("다음 행에 '꽃1' 또는 '꽃2'의 값이 없습니다:")
    print(missing_values)
# '꽃1' 또는 '꽃2' 컬럼에 결측치가 있는 행을 찾습니다.
missing_values = new_df[new_df['꽃1'].isna() | new_df['꽃2'].isna()]

# 결측치가 있는 행이 있는지 확인하고 출력합니다.
if not missing_values.empty:
    print("다음 행에 '꽃1' 또는 '꽃2'의 값이 없습니다:")
    print(missing_values)


In [12]:
# 새로운 데이터프레임에서 꽃1과 꽃2가 동일한 행을 찾습니다.
duplicate_rows = new_df[new_df['꽃1'] == new_df['꽃2']]

# 동일한 행이 있는지 확인합니다.
if not duplicate_rows.empty:
    print("꽃1과 꽃2가 동일한 행이 있습니다.")
    print(duplicate_rows)


In [13]:
duplicate_rows

,꽃1,이미지1,꽃2,이미지2


In [14]:
# '선택한 색상1'과 '선택한 색상2' 열 추가
new_df['선택한 색상1'] = new_df['꽃1'].apply(lambda x: merged_df[merged_df['꽃'] == x]['선택한 색상'].values[0])
new_df['선택한 색상2'] = new_df['꽃2'].apply(lambda x: merged_df[merged_df['꽃'] == x]['선택한 색상'].values[0])

# 열 순서 변경
new_df = new_df[['꽃1', '선택한 색상1', '이미지1', '꽃2', '선택한 색상2', '이미지2']]

# 새로운 데이터 프레임 출력
print(new_df)

               꽃1          선택한 색상1  \
0          문스톤 장미  [247, 243, 204]   
1             산자고  [248, 248, 240]   
2         블랙뷰티 장미      [68, 5, 20]   
3         블랙뷰티 장미      [68, 5, 20]   
4            히야신스    [43, 22, 122]   
..            ...              ...   
827  웨스트민스터 애비 장미  [239, 221, 200]   
828            팬지    [55, 34, 111]   
829       블랙뷰티 장미      [68, 5, 20]   
830            팬지    [55, 34, 111]   
831       피치팡팡 장미   [233, 124, 98]   

                                                  이미지1        꽃2  \
0    https://file.honestflower.kr/media/images/ingr...     스카비오사   
1    https://cdn.cctoday.co.kr/news/photo/201103/59...   로사넬라 장미   
2    https://file.honestflower.kr/media/images/ingr...      흰색동백   
3    https://file.honestflower.kr/media/images/ingr...       옥잠화   
4    https://encrypted-tbn1.gstatic.com/images?q=tb...        수영   
..                                                 ...       ...   
827  https://file.honestflower.kr/media/images/ingr...       설악초   

In [15]:
new_df

,꽃1,선택한 색상1,이미지1,꽃2,선택한 색상2,이미지2
0,문스톤 장미,"[247, 243, 204]",https://file.honestflower.kr/media/images/ingr...,스카비오사,"[218, 196, 217]",https://file.honestflower.kr/media/images/ingr...
1,산자고,"[248, 248, 240]",https://cdn.cctoday.co.kr/news/photo/201103/59...,로사넬라 장미,"[240, 222, 187]",https://file.honestflower.kr/media/images/ingr...
2,블랙뷰티 장미,"[68, 5, 20]",https://file.honestflower.kr/media/images/ingr...,흰색동백,"[255, 255, 255]",https://cdn.011st.com/11dims/resize/600x600/qu...
3,블랙뷰티 장미,"[68, 5, 20]",https://file.honestflower.kr/media/images/ingr...,옥잠화,"[255, 255, 255]",https://yummygarden.co.kr/web/product/big/2019...
4,히야신스,"[43, 22, 122]",https://encrypted-tbn1.gstatic.com/images?q=tb...,수영,"[255, 255, 255]",https://www.cnews.or.kr/news/photo/202108/1121...
...,...,...,...,...,...,...
827,웨스트민스터 애비 장미,"[239, 221, 200]",https://file.honestflower.kr/media/images/ingr...,설악초,"[232, 228, 224]",https://www.nihhs.go.kr/user/AttachFiles/FLOWF...
828,팬지,"[55, 34, 111]",https://encrypted-tbn0.gstatic.com/images?q=tb...,무스카리,"[94, 117, 130]",https://file.honestflower.kr/media/images/ingr...
829,블랙뷰티 장미,"[68, 5, 20]",https://file.honestflower.kr/media/images/ingr...,팬지,"[55, 34, 111]",https://encrypted-tbn0.gstatic.com/images?q=tb...
830,팬지,"[55, 34, 111]",https://encrypted-tbn0.gstatic.com/images?q=tb...,제비꽃,"[96, 57, 161]",https://www.nihhs.go.kr/user/AttachFiles/FLOWF...


6. 최종 합쳐진 데이터 생성

In [16]:
merged_new_df = pd.concat([merged_df.reset_index(drop=True), new_df.reset_index(drop=True)], axis=1)

# 결합된 데이터 프레임 출력
print(merged_new_df)

        꽃   월    일자  계절           꽃말  \
0    각시붓꽃   3   NaN   봄    부끄러움, 세련됨   
1      감국  10   NaN  가을       그윽한 향기   
2     개나리   3   NaN   봄  희망,깊은 정, 달성   
3     개나리   4   1.0   봄           희망   
4    갯개미취   9   9.0  가을           추억   
..    ...  ..   ...  ..          ...   
827   흰앵초   3  26.0   봄          첫사랑   
828  흰제비꽃   1   6.0  겨울     순진무구한 사랑   
829  히아신스   3   NaN   봄   마음의 기쁨, 승리   
830  히야신스   1   4.0  겨울       차분한 사랑   
831   히어리   3   9.0   봄        봄의 노래   

                                                    설명  \
0    각시라 하면 이제 막 시집 온 새색시를 연상케 한다. 그래서인지 여러가지 붓꽃들 중...   
1    가을 산야는 국화과 식물들 차지다. 특히 노란 꽃으로 향기까지 일품인 감국은 보는 ...   
2    우리나라 전역에서 봄 소식을 가장 먼저 알려주는 대표적인 꽃, 개나리는 어렵지 않게...   
3    개나리, 봄을 알리는 전령사 하면 가장 먼저 떠오르는 꽃이다. ‘나리나리 개나리 잎...   
4    옛날 일 따위는 깨끗이 잊는 사람들이 많은 가운데서도 당신은 옛 일을 어제 일처럼 ...   
..                                                 ...   
827  첫사랑의 추억때문에 현실의 사랑을 바라보지 못하고 있지는 않습니까? 아련한 추억은 ...   
828        당신은 내성적이고 고상한 매력이 있지만 은밀한 사랑에 쉽게 빠지는 사람입니다. 

In [17]:
merged_new_df

,꽃,월,일자,계절,꽃말,설명,이미지,색상,선택한 색상,c0,c1,c2,c3,유사 색상,꽃1,선택한 색상1,이미지1,꽃2,선택한 색상2,이미지2
0,각시붓꽃,3,NaN,봄,"부끄러움, 세련됨",각시라 하면 이제 막 시집 온 새색시를 연상케 한다. 그래서인지 여러가지 붓꽃들 중...,https://www.nihhs.go.kr/user/AttachFiles/FLOWF...,보라,"[125, 79, 163]","[251, 250, 205]","[222, 186, 206]","[186, 148, 209]","[127, 102, 157]",c3,문스톤 장미,"[247, 243, 204]",https://file.honestflower.kr/media/images/ingr...,스카비오사,"[218, 196, 217]",https://file.honestflower.kr/media/images/ingr...
1,감국,10,NaN,가을,그윽한 향기,가을 산야는 국화과 식물들 차지다. 특히 노란 꽃으로 향기까지 일품인 감국은 보는 ...,https://www.nihhs.go.kr/user/AttachFiles/FLOWF...,노랑,"[165, 164, 23]","[250, 248, 241]","[250, 234, 177]","[229, 186, 115]","[197, 137, 64]",c3,산자고,"[248, 248, 240]",https://cdn.cctoday.co.kr/news/photo/201103/59...,로사넬라 장미,"[240, 222, 187]",https://file.honestflower.kr/media/images/ingr...
2,개나리,3,NaN,봄,"희망,깊은 정, 달성","우리나라 전역에서 봄 소식을 가장 먼저 알려주는 대표적인 꽃, 개나리는 어렵지 않게...",https://file.honestflower.kr/media/images/ingr...,노랑,"[213, 182, 54]","[28, 10, 0]","[54, 21, 0]","[96, 54, 1]","[204, 149, 68]",c3,블랙뷰티 장미,"[68, 5, 20]",https://file.honestflower.kr/media/images/ingr...,흰색동백,"[255, 255, 255]",https://cdn.011st.com/11dims/resize/600x600/qu...
3,개나리,4,1.0,봄,희망,"개나리, 봄을 알리는 전령사 하면 가장 먼저 떠오르는 꽃이다. ‘나리나리 개나리 잎...",https://file.honestflower.kr/media/images/ingr...,노랑,"[214, 182, 54]","[28, 10, 0]","[54, 21, 0]","[96, 54, 1]","[204, 149, 68]",c3,블랙뷰티 장미,"[68, 5, 20]",https://file.honestflower.kr/media/images/ingr...,옥잠화,"[255, 255, 255]",https://yummygarden.co.kr/web/product/big/2019...
4,갯개미취,9,9.0,가을,추억,옛날 일 따위는 깨끗이 잊는 사람들이 많은 가운데서도 당신은 옛 일을 어제 일처럼 ...,https://lh6.googleusercontent.com/proxy/yRZlN4...,보라,"[209, 174, 248]","[12, 19, 79]","[29, 38, 125]","[92, 70, 156]","[212, 173, 252]",c3,히야신스,"[43, 22, 122]",https://encrypted-tbn1.gstatic.com/images?q=tb...,수영,"[255, 255, 255]",https://www.cnews.or.kr/news/photo/202108/1121...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
827,흰앵초,3,26.0,봄,첫사랑,첫사랑의 추억때문에 현실의 사랑을 바라보지 못하고 있지는 않습니까? 아련한 추억은 ...,https://digitalchosun.dizzo.com/site/data/img_...,초록,"[208, 224, 221]","[252, 209, 209]","[236, 226, 225]","[211, 224, 220]","[174, 225, 225]",c2,웨스트민스터 애비 장미,"[239, 221, 200]",https://file.honestflower.kr/media/images/ingr...,설악초,"[232, 228, 224]",https://www.nihhs.go.kr/user/AttachFiles/FLOWF...
828,흰제비꽃,1,6.0,겨울,순진무구한 사랑,당신은 내성적이고 고상한 매력이 있지만 은밀한 사랑에 쉽게 빠지는 사람입니다.,https://img1.daumcdn.net/thumb/R1280x0.fjpg/?f...,흰색,"[236, 238, 230]","[28, 103, 88]","[61, 131, 97]","[214, 205, 164]","[238, 242, 230]",c3,팬지,"[55, 34, 111]",https://encrypted-tbn0.gstatic.com/images?q=tb...,무스카리,"[94, 117, 130]",https://file.honestflower.kr/media/images/ingr...
829,히아신스,3,NaN,봄,"마음의 기쁨, 승리",꽃에는 늘 전설이 있게 마련이다. 히야신스는 그리이스의 아름다운 청년 Hyacint...,https://upload.wikimedia.org/wikipedia/commons...,보라,"[121, 31, 240]","[0, 0, 0]","[82, 5, 123]","[137, 44, 220]","[188, 111, 241]",c2,블랙뷰티 장미,"[68, 5, 20]",https://file.honestflower.kr/media/images/ingr...,팬지,"[55, 34, 111]",https://encrypted-tbn0.gstatic.com/images?q=tb...
830,히야신스,1,4.0,겨울,차분한 사랑,봄이 찾아올 때마다 포근한 향기와 함께 피는 히야신스를 탄생화로 갖는 당신은 상냥하...,https://encrypted-tbn1.gstatic.com/images?q=tb...,보라,"[43, 22, 122]","[12, 19, 79]","[29, 38, 125]","[92, 70, 156]","[212, 173, 252]",c1,팬지,"[55, 34, 111]",https://encrypted-tbn0.gstatic.com/images?q=tb...,제비꽃,"[96, 57, 161]",https://www.nihhs.go.kr/user/AttachFiles/FLOWF...


In [18]:
# 조건 확인: merged_new_df에서 [꽃] == [꽃1] 또는 [꽃] == [꽃2]가 있는지 확인
condition = (merged_new_df['꽃'] == merged_new_df['꽃1']) | (merged_new_df['꽃'] == merged_new_df['꽃2'])
matching_rows = merged_new_df[condition]

In [19]:
matching_rows

,꽃,월,일자,계절,꽃말,설명,이미지,색상,선택한 색상,c0,c1,c2,c3,유사 색상,꽃1,선택한 색상1,이미지1,꽃2,선택한 색상2,이미지2


7. csv 변환

In [20]:
# merged_new_df를 CSV 파일로 저장
merged_new_df.to_csv('/content/drive/MyDrive/TAVE프로젝트/data/어울리는색상데이터.csv', index=False)
